# Answering Business Questions using SQL

## Introduction and Schema Diagram

In this project we'll work with sort of databases that most businesses use. The [Chinook database](https://github.com/lerocha/chinook-database) contains information about a fictional digital music shop - kind of like a mini-iTunes store.

The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in 11 tables. A schema diagram will help us understand the available columns and the structure of data. Here's a schema diagram for the Chinook database:
![schema](./chinook_schema.png)

In [104]:
### Connect to database
# %%capture
%load_ext sql
%sql sqlite:///chinook.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @chinook.db'

## Overview of the Data

Let's start by getting familiar with our data. 

In [105]:
%%sql
SELECT name, type FROM sqlite_master
WHERE type IN ("table", "view")

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [106]:
%%sql
SELECT * FROM track
LIMIT 5

 * sqlite:///chinook.db
Done.


track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",230619,3990994,0.99
4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman",252051,4331779,0.99
5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In [107]:
%%sql
SELECT * FROM customer
LIMIT 5

 * sqlite:///chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


## Selecting Albums to Purchase

We're interested in finding out which genres sell the best in the USA.

In [108]:
%%sql
/* ### Query sales count and percentage by genres */
WITH sales_genres AS 
    (
    SELECT genre.name, COUNT(*) AS sales FROM track
    LEFT JOIN invoice_line ON track.track_id = invoice_line.track_id
    LEFT JOIN genre ON track.genre_id = genre.genre_id
    GROUP BY track.genre_id
    ORDER BY sales DESC 
    )
    
SELECT *, ROUND(sales*100.0 / (SELECT TOTAL(sales)*1.0 
                        FROM sales_genres), 2) 
                        AS percentage                
FROM sales_genres

 * sqlite:///chinook.db
Done.


name,sales,percentage
Rock,3017,46.75
Metal,755,11.7
Alternative & Punk,648,10.04
Latin,627,9.71
Jazz,190,2.94
R&B/Soul,165,2.56
Blues,149,2.31
Alternative,123,1.91
Classical,105,1.63
TV Shows,93,1.44


Based on the sales of tracks across different genres in the USA, we see that `Rock`, `Metal`, `Alternative & Punk` and `Latin` are the best selling and occupy about 78% of the market. 

## Analyzing Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We'll analyze the purchases of customer belonging to each employee to see if any sales support agent is performing either better or worse that others.

In [109]:
%%sql
/* ### Query total amount of sales by each sales support agent */
SELECT em.first_name, em.last_name, em.title, em.hire_date, ROUND(SUM(invoice.total)) AS amount 
FROM employee AS em
LEFT JOIN customer ON employee_id = support_rep_id
LEFT JOIN invoice ON customer.customer_id = invoice.customer_id
WHERE em.title = 'Sales Support Agent'
GROUP BY em.employee_id

 * sqlite:///chinook.db
Done.


first_name,last_name,title,hire_date,amount
Jane,Peacock,Sales Support Agent,2017-04-01 00:00:00,1732.0
Margaret,Park,Sales Support Agent,2017-05-03 00:00:00,1584.0
Steve,Johnson,Sales Support Agent,2017-10-17 00:00:00,1394.0


There is a 20% difference in sales between the top and the bottom employee, which corresponds with the differences in their hiring dates.

## Analyzing Sales by Country

Next task is to analyze the sales data for customers from each country. We should use the country value from `customers` table, not the `invoice` table.

In [110]:
%%sql
WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT COUNT(*)
                 FROM customer
                 WHERE country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice AS i ON i.invoice_id = il.invoice_id
     INNER JOIN customer AS c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        COUNT(DISTINCT customer_id) AS customers,
        ROUND(SUM(unit_price)) AS total_sales,
        ROUND(SUM(unit_price) / COUNT(distinct customer_id), 2) AS customer_lifetime_value,
        ROUND(SUM(unit_price) / COUNT(distinct invoice_id), 2) AS average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, customer_lifetime_value DESC
    )

 * sqlite:///chinook.db
Done.


country,customers,total_sales,average_order,customer_lifetime_value
Czech Republic,2,273.0,9.11,136.62
Portugal,2,185.0,6.38,92.57
India,2,183.0,8.72,91.58
Brazil,5,428.0,7.01,85.54
Germany,4,335.0,8.16,83.66
United Kingdom,3,246.0,8.77,81.84
USA,13,1040.0,7.94,80.04
France,5,389.0,7.78,77.81
Canada,8,536.0,7.05,66.95
Other,15,1095.0,7.45,73.0


Based on the data, the most valuable customers live in the countries such as:

- Czech Republic
- Portugal
- India

It's worth keeping in mind that because the amount of data from each of these countries is relatively low.  Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence.  A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.

## Albums vs Individual Tracks

We're going to identify whether each invoice has all the tracks from an album. This can be useful for choosing purchasing strategy purposes.

In [111]:
%%sql
/* ### Query that categorizes each invoice as either an album purchase or not and calculates summary statistics */
WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id AS invoice_id,
         MIN(il.track_id) AS first_track_id
     FROM invoice_line AS il
     GROUP BY invoice_id
    )

SELECT
    album_purchase,
    COUNT(invoice_id) AS number_of_invoices,
    ROUND(100.0*COUNT(invoice_id) / (SELECT COUNT(*) FROM invoice), 2) || "%" AS percent
FROM
    (
    SELECT ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track AS t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track AS t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line AS il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
        
        
                 AND
        
        
                 (
                  SELECT il2.track_id FROM invoice_line AS il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track AS t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track AS t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS album_purchase
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase

 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percent
no,500,81.43%
yes,114,18.57%


Album purchases account for 18.6% of purchases.  

## Conclusions

We've explored the available customer data using SQL queries and come up with a genre, country and album data that we can provide to marketing for idea generation. We have identified facts such as:

- `Rock` genre makes up half of the market


- `Rock`, `Metal`, `Alternative & Punk` and `Latin` are the best selling and together occupy about 78% of the market


- Employee sales performance is pretty uniform and depends on time spent on work


- The most valuable customers live in the Czech Republic, Portugal, India


- Album purchases account for 18.6% of purchases